In [18]:
from selenium.webdriver.common.by import By
from tqdm.auto import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import os
import re
import spacy
import datetime
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import itertools

In [5]:
# Если не с качивается модель, то попробовать вот так
# !pip install spacy
# !pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.4.0/ru_core_news_sm-3.4.0.tar.gz
# скачивание модели на локальный диск
# nlp.to_disk('ru_core_news_sm-3.4.0')
# nlp = spacy.load('ru_core_news_sm-3.4.0')
# pip freeze > requirements.txt

# Функции

In [6]:
def lemm_finder(list_of_text, stop_words=0):
    """ Функция позволяет перевети все формы слова в леммы"""
    lemma_list = []
    for i in tqdm(list_of_text):
        text = ' '.join(re.findall(r'[А-Яа-я]+', i))
        doc = nlp(text)
        lemma = []
        for token in doc:
            lemma.append(token.lemma_)
        text_new = ' '.join(lemma)
        if stop_words != 0:
            for j in stop_words:
                text_new = re.sub(rf'( {j}\b)', '', text_new)
        lemma_list.append(text_new)
    return lemma_list


def test_page(driver, delay, element):
    '''Функция проверки загрузки страницы
        delay - количесво секунд ожидания
        element - что ищем пример (By.CLASS_NAME, 'popup-obj')
        driver - драйвер пример
        if __name__ == "__main__":
        driver = Chrome(executable_path="./chromedriver.exe")'''
    # проверка загрузилась ли стр
    try:
        myElem = WebDriverWait(driver, delay).until(
            EC.presence_of_element_located(
                element))
    except TimeoutException:
        print("Loading took too much time!")


# запускае браузер
def get_screens(city_list, url='https://lostarmour.info/map/?ysclid=latojsz8nr362636823)', Date='29_11_22'):
    # Настрока и запуск браузера
    options = Options()
    options.page_load_strategy = 'normal'
    driver = webdriver.Chrome(executable_path="./chromedriver.exe", options=options)
    # Переход на сайт
    driver.get(url)
    # Переключаемся на iframe
    driver.switch_to.frame(driver.find_element(By.TAG_NAME, "iframe"))
    window_before = driver.window_handles[0]
    # Переходим на страницу яндекс карт
    try:
        driver.find_element(By.XPATH,
                            "/html/body/div/div/div/div[2]/div[1]/div[1]/div[4]/div/a").click()
    except:
        driver.find_element(By.XPATH,
                            "/html/body/div/div/div/div[2]/div[1]/div[1]/div[4]/div/a").click()
    window_after = driver.window_handles[1]
    # Переключаемся обратно
    driver.switch_to.default_content()
    driver.switch_to.window(window_after)
    # Экран в полный формат
    driver.fullscreen_window()

    for city in tqdm(city_list):
        try:
            # Ввод тектса
            driver.find_element(By.TAG_NAME, "input").send_keys(city)
            # Нажать на ввод поиска
            driver.find_element(By.TAG_NAME, "button").click()
            # Цикл для зума
            for i in range(10):
                time.sleep(0.1)
                # Нажать на кнопку приближения
                driver.find_element(By.CLASS_NAME, "zoom-control__zoom-in").click()
            # Проверяем загрузился ли элемент
            test_page(driver=driver, delay=3,
                      element=(By.XPATH,
                               "//span[@class='inline-image _loaded sidebar-toggle-button__icon']"))
            time.sleep(1)
            # Нажать на кнопку для того, чтобы убрать поиск
            driver.find_element(
                By.XPATH,
                "//span[@class='inline-image _loaded sidebar-toggle-button__icon']").click()
            # Создаем папку для фото
            if not os.path.exists(f"Скриншоты/{Date}"):
                os.mkdir(f"Скриншоты/{Date}")

            # Делаем скриншот
            driver.save_screenshot(f'Скриншоты/{Date}/{city}.png')
            # Отчистить поле поиска нажав кнопку
            driver.find_element(
                By.XPATH,
                "//div[@class='small-search-form-view__icon _type_close']").click()
        except:
            print(f'Проблема с городом {city}')
            continue
    driver.quit()


def find_words(key_words, text):
    """Проверяет есть ли слова в списке текстов, если есть, то возвращает их, если нет то
    возвращает 0"""
    find_list = []
    for i in key_words:
        if len(re.findall(rf'{i}', text)) != 0:
            find_list = find_list + re.findall(rf'{i}', text)
    find_list = ', '.join(find_list)
    return find_list


def find_distance(sentence, word1, word2):
    """Находит колличесво слов между двумя словами в тексте"""
    distances = []
    while sentence != "":
        _, _, sentence = sentence.partition(word1)
        text, _, _ = sentence.partition(word2)
        if text != "":
            distances.append(len(text.split()))
    return distances

def text_cheсk(df_with_text, text_column_name, all_key_words, cities_list):
    """определеяет в каких текстах есть ключивые слова (пересечение городов и проблеммы)"""
    df_texts = df_with_text.copy()
    # Выделяем леммы
    new_text_list = lemm_finder(list_of_text=list(df_texts[text_column_name]))
    # Проверяем тексты на ключевые слова
    df_texts["New_text"] = new_text_list
    df_texts["Contains_key_words"] = df_texts['New_text'].str.contains('|'.join(all_key_words)).astype(int)
    df_texts["Contains_cities_list"] = df_texts['New_text'].str.contains('|'.join(cities_list)).astype(int)

    # Разделяем ключевые словосочетания на слова
    all_key_words_split = []
    for i in all_key_words:
        all_key_words_split = all_key_words_split + i.split()
    all_key_words_split = list(set(all_key_words_split))
    # Формируем все возможные комбинации названий городов и ключевых слов
    a = [cities_list,all_key_words_split]
    all_combinations = list(itertools.product(*a))

    good_index = []
    for text_index in df_texts.index:
        for iteration in range(len(all_combinations)):
            try :
                word_distance = find_distance(df_texts.loc[text_index]['New_text'], all_combinations[iteration][0], all_combinations[iteration][1])[0]
                if word_distance <= 20:
                    good_index.append(text_index)
            except:
                continue
    good_index = set(good_index)
    df_texts["Contains_20_words_length"] = [ 1 if a in good_index else 0  for a in df_texts.index]
    df_texts["Contains_sum"] = df_texts["Contains_cities_list"] + df_texts["Contains_key_words"] + df_texts["Contains_20_words_length"]
    df_with_text['contain_key_words'] = contain_key_words_list = [1 if a == 3 else 0 for a in df_texts["Contains_sum"]]
    df_with_text['key_words_find'] = [
        find_words(all_key_words + cities_list, df_texts.loc[index]['New_text']) if df_texts.loc[index][
                                                                                        'Contains_sum'] == 3 else 0 for
        index in df_texts.index]
    return df_with_text

# Код

In [7]:
# Определяем сегодняшнюю дату
now = datetime.datetime.now()
Date = now.strftime("%d_%m_%Y")
# собираем лист название городов
df_cities = pd.read_excel('Входные данные/Список_населенных_пунктов.xlsx')
cities_list = df_cities['Населенные_пункты'].dropna()
get_screens(city_list=cities_list, url='https://lostarmour.info/map/?ysclid=latojsz8nr362636823)', Date=Date)

/var/folders/pt/rpf8lgcj6zjc0wsmcg9jyy6r0000gn/T/ipykernel_5279/1331623.py:39: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="./chromedriver.exe", options=options)


  0%|          | 0/29 [00:00<?, ?it/s]

Проблема с городом Мариуполь
Проблема с городом Славянск
Проблема с городом Горловка
Проблема с городом Ясиноватая
Проблема с городом Угледар
Проблема с городом Запорожье
Проблема с городом Энергодар
Проблема с городом Краснодар
Проблема с городом Новороссийск
Проблема с городом Сочи
Проблема с городом Анапа
Проблема с городом Курск
Проблема с городом Белгород


In [162]:
#  Блок Тимофея
#russian_regions = ['крым', 'симферополь', 'севастополь', 'краснодарский край', 'сочи', 'белгород',
#                    'белгородская область', 'ростов', 'ростовская область', 'курская область', 'брянская область']

In [19]:
# Скачиваем данные
df_texts0 = pd.read_excel('Входные данные/Тексты.xlsx')
# Загружаем модель для обработки русского текста
nlp = spacy.load('ru_core_news_sm-3.4.0')
df_cities = pd.read_excel('Входные данные/Список_населенных_пунктов.xlsx')
# собираем лист название городов
cities_list = lemm_finder(list(df_cities['Населенные_пункты'].dropna()), stop_words=['область', 'обл', 'край', 'обл.'])

# Выявление сообщений с ключевыми словами и городами
text_cheсk_df = text_cheсk(df_with_text=df_texts0, text_column_name='Выдержки из текста',
                           all_key_words=lemm_finder(list(df_cities['Тип_проблемы'].dropna())), cities_list=cities_list)

text_cheсk_df.to_excel(f'Результат_обработки/{Date}.xlsx')


  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

In [29]:
import Zones

In [33]:

Zones.get_territory_status(list(text_cheсk_df['key_words_find']),result_column)

[ WARN:0@2188.198] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('Скриншоты/29_11_22/Запорожская обл..png'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [51]:
import os
import cv2
import numpy as np
import math
import datetime

now = datetime.datetime.now()
Date = now.strftime("%d_%m_%Y")

def get_image_path(image_name: str):
    now = datetime.datetime.now()
    Date = now.strftime("%d_%m_%Y")

    image_path = f'Скриншоты/{Date}' + '/' + image_name

    return image_path


def get_image_colors(image_name: str):
    image_path = get_image_path(image_name)
    image = cv2.imread(image_path)

    image_colors = []

    for i in (1, 2, 3, 4):
        gbr_colors = image[image.shape[0] // i - 1, image.shape[1] // i - 1]
        image_colors.append(gbr_colors[::-1])

    return image_colors


def calculate_color_differences_percent(first_color: list, second_color: list):
    maximum_difference = math.sqrt(3 * 256 ** 2)

    color_difference = math.sqrt((first_color[0] - second_color[0]) ** 2 + (first_color[1] - second_color[1]) ** 2 + (
                first_color[2] - second_color[2]) ** 2)

    color_difference_percent = color_difference / maximum_difference * 100

    return color_difference_percent


def check_colors(image_colors: list, zone_colors: list):
    count = 0
    for i in image_colors:
        for j in zone_colors:
            if calculate_color_differences_percent(i, j) < 6.00:
                count += 1

    if count >= 2:
        return True
    else:
        return False


def get_zone_name(image_colors: list):
    orange_zone_colors = [[250, 223, 186], [204, 200, 182], [223, 213, 149]]
    red_zone_colors = [[196, 177, 190], [242, 200, 194], [215, 190, 156]]

    if check_colors(image_colors, orange_zone_colors):
        return 'В зоне ДНР/ЛНР'
    elif check_colors(image_colors, red_zone_colors):
        return 'В зоне ДНР/ЛНР'
    else:
        return 'Не в зоне ДНР/ЛНР'





def get_territory_status(key_words_find: list, result_column: dict):
    territory_status = []
    for i in key_words_find:
        if i == 0:
            territory_status.append(0)
        else:
            row_list = i.split(', ')
            res_list = []
            for j in row_list:
                try:
                    result_column[j]
                except KeyError:
                    pass
                else:
                    res_list.append(result_column[j])

            territory_status.append(res_list)

    return territory_status


In [52]:
result_column = {}
image_names = os.listdir(f'Скриншоты/{Date}')

for image_name in image_names:
    image_colors = get_image_colors(image_name)
    zone_name = get_zone_name(image_colors)
    result_column[image_name.lower()[:-4]] = zone_name

print(result_column)

{'донецк': 'В зоне ДНР/ЛНР', 'горловка': 'В зоне ДНР/ЛНР', 'запорожская обл.': 'В зоне ДНР/ЛНР', 'новороссийск': 'Не в зоне ДНР/ЛНР', 'ростовская обл.': 'Не в зоне ДНР/ЛНР', 'ясиноватая': 'В зоне ДНР/ЛНР', 'крым': 'Не в зоне ДНР/ЛНР', 'мариуполь': 'В зоне ДНР/ЛНР', 'анапа': 'Не в зоне ДНР/ЛНР', 'донбасс': 'Не в зоне ДНР/ЛНР', 'днр': 'В зоне ДНР/ЛНР', 'севастополь': 'Не в зоне ДНР/ЛНР', 'курск': 'Не в зоне ДНР/ЛНР', 'курская область': 'Не в зоне ДНР/ЛНР', 'херсонская обл.': 'В зоне ДНР/ЛНР', 'энергодар': 'В зоне ДНР/ЛНР', 'угледар': 'Не в зоне ДНР/ЛНР', 'брянская область': 'Не в зоне ДНР/ЛНР', 'геообъекты': 'Не в зоне ДНР/ЛНР', 'запорожье': 'В зоне ДНР/ЛНР', 'донецкая область': 'В зоне ДНР/ЛНР', 'краснодар': 'Не в зоне ДНР/ЛНР', 'белгородская область': 'Не в зоне ДНР/ЛНР', 'лнр': 'В зоне ДНР/ЛНР', 'краснодарский край': 'Не в зоне ДНР/ЛНР', 'славянск': 'В зоне ДНР/ЛНР', 'белгород': 'Не в зоне ДНР/ЛНР', 'луганская область': 'В зоне ДНР/ЛНР'}


In [42]:
result_column.get('белгород')

'Не в зоне ДНР/ЛНР'

In [54]:

text_cheсk_df['territory_status'] = get_territory_status(list(text_cheсk_df['key_words_find']), result_column)

text_cheсk_df

,Подзаголовок,Дата,Источник,Заголовок,Выдержки из текста,Ссылка на источник,Автор,Категория источника,Уровень источника,Регион источника,Индекс заметности,Охват аудитории,Похожих публикаций (с учетом выбранной группировки),contain_key_words,key_words_find,territory_status
0,NaN,17.11.2022 22:09,Комсомольская правда - Белгород (bel.kp.ru),Энергетики восстановили электроснабжение больш...,В настоящее время в зоне отключения остаются...,https://www.bel.kp.ru/online/news/5015379/,NaN,Порталы СМИ,Региональный,"Белгород (город, Белгородская Область)",301,41417,2,0,0,0
1,NaN,17.11.2022 22:01,Царьград (md.tsargrad.tv). Молдова,Украина осталась без ракетных двигателей – уда...,...,https://md.tsargrad.tv/news/ukraina-ostalas-be...,NaN,Порталы СМИ,Зарубежный,Молдавия,102,14646,2,0,0,0
2,NaN,17.11.2022 21:49,Красная весна (rossaprimavera.ru),Укрэнерго: 70% потребителей центра Украины вер...,...,https://rossaprimavera.ru/news/75525e24,NaN,Ленты информагентств,Федеральный,Москва,27,138955,1,0,0,0
3,NaN,17.11.2022 21:27,Курс дела (kursdela.biz),Челябинская область переходит на ГИС ЖКХ: что ...,"Предполагается, что большинство вопросов мож...",https://kursdela.biz/news/2022-11-17/chelyabin...,NaN,Порталы СМИ,Региональный,"Челябинск (город, Челябинская Область)",8,4359,1,0,0,0
4,NaN,17.11.2022 21:20,Комсомольская правда (kp.ru),"На Украине будет не только темно, но и холодно",ЕС констатирует: Разрушение энергетики Украи...,https://www.kp.ru/daily/27472/4679184/,NaN,Порталы СМИ,Федеральный,Москва,376,2070504,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,NaN,17.11.2022 02:45,Sololaki (blog.sololaki.ru),Поставки нефти почему-то остановились только в...,"Что с ""Дружбой""? Фьючерсы на нефть марки W...",https://sololaki.ru/postavki-nefti-pochemu-to-...,NaN,Порталы СМИ,Зарубежный,Грузия,1,2,1,0,0,0
105,NaN,17.11.2022 01:44,Ежедневные новости Владивостока (novostivl.ru),Небензя отверг попытки возложить на РФ ответст...,17 ноября - NVL. Постпред при ООН...,https://novostivl.ru/news/20221117/82746/,NaN,Порталы СМИ,Региональный,"Владивосток (город, Приморский Край)",14,369716,1,0,0,0
106,NaN,17.11.2022 01:43,Ежедневные новости Владивостока (novostivl.ru),RusVesna проинформировала о взрывах в Джанкое ...,17 ноября - NVL. В северной части...,https://novostivl.ru/news/20221117/82748/,NaN,Порталы СМИ,Региональный,"Владивосток (город, Приморский Край)",28,369716,2,0,0,0
107,NaN,17.11.2022 00:02,Волгодонская правда (v-pravda.ru),Ситуация на Украине и в Донбассе. 17 ноября. Х...,скриншот видео Минобороны РФ Вооружен...,https://v-pravda.ru/2022/11/17/situaciya-na-uk...,NaN,Порталы СМИ,Региональный,"Волгодонск (город, Ростовская Область)",9,3155,1,0,0,0
